In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm

In [10]:
df = pd.read_csv('../data/cleaned/cleaned_vessel_calls.csv')
df = df.drop(columns=['vessel.vesselName','Timestamp'])

In [11]:
df

,arrivalDate,sailingDate,place.placeName,place.unlocode,place.placeType,place.countryName,Event_Type,IMO,vessel.vesselType
0,2021-01-17T10:08:08,2021-01-20T03:44:00,Longkou Port,CNLKU,Port,People's Republic of China,ARRIVAL,9337391,crude oil tanker
1,2021-01-17T10:08:08,2021-01-20T03:44:00,Longkou Port,CNLKU,Port,People's Republic of China,DEPARTURE,9337391,crude oil tanker
2,2021-01-25T05:16:08,2021-01-27T02:04:05,Longkou Port,CNLKU,Port,People's Republic of China,ARRIVAL,9337391,crude oil tanker
3,2021-01-25T05:16:08,2021-01-27T02:04:05,Longkou Port,CNLKU,Port,People's Republic of China,DEPARTURE,9337391,crude oil tanker
4,2021-02-21T01:17:30,2021-02-22T23:00:19,Zhoushan Port,CNZOS,Port,People's Republic of China,ARRIVAL,9337391,crude oil tanker
...,...,...,...,...,...,...,...,...,...
344421,2023-08-25T14:44:08,2023-08-29T17:06:17,Leixoes,PTLEI,Port,Portugal,DEPARTURE,9336787,bulk carrier
344422,2023-09-14T22:22:15,2023-09-19T13:32:02,Gent (Ghent),BEGNE,Port,Belgium,ARRIVAL,9336787,bulk carrier
344423,2023-09-14T22:22:15,2023-09-19T13:32:02,Gent (Ghent),BEGNE,Port,Belgium,DEPARTURE,9336787,bulk carrier
344424,2023-09-22T11:25:42,2023-09-28T01:32:27,la Pallice,FRLPE,Sub Port,France,ARRIVAL,9336787,bulk carrier


In [3]:
df["arrivalDate"] = pd.to_datetime(df["arrivalDate"])
df["sailingDate"] = pd.to_datetime(df["sailingDate"])

In [4]:
df.sort_values(by=["IMO", "arrivalDate"], inplace=True)
df['next_unlocode'] = df.groupby('IMO')['place.unlocode'].shift(-1)
df['is_last_imo'] = df['IMO'] != df['IMO'].shift(-1)
df_cleaned = df[~df["is_last_imo"]]

In [ ]:
df

In [5]:
df_cleaned.drop(columns=['is_last_imo'])
df_cleaned = df_cleaned[df_cleaned["Event_Type"] != "ARRIVAL"]

In [6]:
df_cleaned['duration'] = (df_cleaned["sailingDate"] - df_cleaned["arrivalDate"]).dt.total_seconds() / 3600

In [8]:
data = df_cleaned[['duration', 'place.unlocode', 'place.placeType', 'place.countryName', 'vessel.vesselType', 'next_unlocode']]
data = data.rename(columns={'place.unlocode':'unlocode', 'vessel.vesselType':'vesselType', 'place.placeType': 'placeType', 'place.countryName': 'countryName'})

In [ ]:
df